<a href="https://colab.research.google.com/github/jiyanshud22/Affective-Computing/blob/main/Code_for_AR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input Floors

In [ ]:
default_floors = [
    {"name": "L001Ground Floor", "usable_area": 1595, "max_loading": 160},
    {"name": "L002Floor 01", "usable_area": 1181, "max_loading": 120},
    {"name": "L003Floor 02", "usable_area": 2363, "max_loading": 225},
    {"name": "L004Floor 03", "usable_area": 2363, "max_loading": 225},
    {"name": "L005Floor 04", "usable_area": 2363, "max_loading": 225},
    {"name": "L006Floor 05", "usable_area": 2363, "max_loading": 225},
    {"name": "L007Floor 06", "usable_area": 2363, "max_loading": 225},
    {"name": "L008Floor 07", "usable_area": 2327, "max_loading": 225},
    {"name": "L009Floor 08", "usable_area": 2281, "max_loading": 210},
]

def get_floor_data(num_floors):
    floors = []
    for i in range(num_floors):
        if i < len(default_floors):
            print(f"\nFloor {i} ({default_floors[i]['name']}):")
            print(f"Default Usable Area (SQM): {default_floors[i]['usable_area']}")
            print(f"Default Max Assignable Floor Loading Capacity: {default_floors[i]['max_loading']}")
            name = default_floors[i]['name']
            usable_area = default_floors[i]['usable_area']
            max_loading = default_floors[i]['max_loading']
        else:
            print(f"\nFloor {i}:")
            name = f"L00{i+1}Floor {i:02d}"  # Default name for new floors
            usable_area = 0.0
            max_loading = 0.0

        floors.append({
            "name": name,
            "usable_area": usable_area,
            "max_loading": max_loading
        })

    print("\nCollected Floor Data (as of 02:19 PM IST, Thursday, May 15, 2025):")
    print("Name\t\tUsable Area (SQM)\tMax Assignable Floor Loading Capacity")
    for floor in floors:
        print(f"{floor['name']}\t{floor['usable_area']}\t\t\t{floor['max_loading']}")

    return floors

# Usage example
if __name__ == "__main__":
    num_floors_to_collect = 9  # or any number you want
    all_floors = get_floor_data(num_floors_to_collect)



Floor 0 (L001Ground Floor):
Default Usable Area (SQM): 1595
Default Max Assignable Floor Loading Capacity: 160

Floor 1 (L002Floor 01):
Default Usable Area (SQM): 1181
Default Max Assignable Floor Loading Capacity: 120

Floor 2 (L003Floor 02):
Default Usable Area (SQM): 2363
Default Max Assignable Floor Loading Capacity: 225

Floor 3 (L004Floor 03):
Default Usable Area (SQM): 2363
Default Max Assignable Floor Loading Capacity: 225

Floor 4 (L005Floor 04):
Default Usable Area (SQM): 2363
Default Max Assignable Floor Loading Capacity: 225

Floor 5 (L006Floor 05):
Default Usable Area (SQM): 2363
Default Max Assignable Floor Loading Capacity: 225

Floor 6 (L007Floor 06):
Default Usable Area (SQM): 2363
Default Max Assignable Floor Loading Capacity: 225

Floor 7 (L008Floor 07):
Default Usable Area (SQM): 2327
Default Max Assignable Floor Loading Capacity: 225

Floor 8 (L009Floor 08):
Default Usable Area (SQM): 2281
Default Max Assignable Floor Loading Capacity: 210

Collected Floor Data (a

# Input Blocks

In [ ]:
import pandas as pd
import json

# Load the Excel file
file_path = "/content/Auto Stacking Input New Build - Case Study A- R1 (with block instances) (2).xlsx"

# Read the sheet with header on row 2 (index 1)
df = pd.read_excel(file_path, sheet_name='Program Table Input 1 - Block', header=1)

# Drop completely empty rows
df.dropna(how='all', inplace=True)

# Drop the 'Block Id' column if it exists
if 'Block ID' in df.columns:
    df.drop(columns=['Block ID'], inplace=True)

# Convert remaining columns to a list of dictionaries
all_block_data = df.to_dict(orient='records')

# Optional: print or save as JSON
print(json.dumps(all_block_data, indent=2))


[
  {
    "Unnamed: 0": NaN,
    "Individual Zone Work Desks (no partition)": "Individual Zone Work Desks (no partition)",
    "Individual Zone Work Desks (no partition) 1": "Individual Zone Work Desks (no partition) 2",
    "Unnamed: 3": NaN,
    "Work Zone": "Work Zone",
    "Anchor Resident_Anchor Resident": "Anchor Resident_Anchor Resident",
    "Anchor Resident": "Anchor Resident",
    "Anchor Resident.1": "Anchor Resident",
    "NA": NaN,
    "3": 3,
    "1": 1,
    "1.1": 1,
    "ME": "ME",
    "Typical": "Typical",
    "3.1": 3.0,
    "1.77": 1.77,
    "Open": "Open",
    "4.77": 4.77,
    "1.2": 1.0,
    "1.3": 1.0,
    "1.4": 1.0
  },
  {
    "Unnamed: 0": NaN,
    "Individual Zone Work Desks (no partition)": "Individual Zone Work Desks (no partition)",
    "Individual Zone Work Desks (no partition) 1": "Individual Zone Work Desks (no partition) 3",
    "Unnamed: 3": NaN,
    "Work Zone": "Work Zone",
    "Anchor Resident_Anchor Resident": "Anchor Resident_Anchor Resident",
 

# Destination

In [ ]:
floors_data = all_floors
# Filter blocks of type "Destination" from all_block_data
destination_blocks = [
    block for block in all_block_data
    if str(block.get("Typical/Destination", "")).strip().lower() in ["destination", "both"]
]



for floor in all_floors:
    floor["assigned_blocks"] = []
    floor["available_area"] = floor["usable_area"]
    floor["available_loading"] = floor["max_loading"]



# Prioritize community blocks
community_blocks = [b for b in destination_blocks if b.get("Block Type") == "Community - Visitor Experience"]
other_blocks = [b for b in destination_blocks if b.get("Block Type") != "Community - Visitor Experience"]

# Assign blocks to floors
unassigned_blocks = []

def assign_blocks_to_floors(blocks, floors):
    unassigned = []
    for block in blocks:
        placed = False
        for floor in floors:
            area_needed = block.get("Cumulative Block + Circulation Area (SQM)", 0)
            loading_needed = block.get("Max Occupancy by Block", 0) * block.get("Block Capacity",0)
            if area_needed <= floor["available_area"] and loading_needed <= floor["available_loading"]:
                floor["assigned_blocks"].append(block)
                floor["available_area"] -= area_needed
                floor["available_loading"] -= loading_needed
                placed = True
                break
        if not placed:
            unassigned.append(block)
    if unassigned:
        print("\n🚫 Unassigned Blocks:")
        for block in unassigned:
             print(f" - {block['Block Name']} | Area: {block.get('Cumulative Block + Circulation Area (SQM)', 0)} | Occupancy: {block.get('Max Occupancy by Block', 0)}")
    else:
        print("\n✅ All blocks assigned successfully.")



    return unassigned

# Assign community blocks first, starting from ground floor
unassigned_community = assign_blocks_to_floors(community_blocks, all_floors)

# Assign other destination blocks
unassigned_others = assign_blocks_to_floors(other_blocks, all_floors)

# Combine unassigned blocks
unassigned_blocks = unassigned_community + unassigned_others

# Prepare output
assignment_summary = [{
    "floor": floor["name"],
   # "assigned_blocks": [block["Block Name"] for block in floor["assigned_blocks"]],
    "max_loading": floor["max_loading"],
    "usable_area": floor["usable_area"],
    "used_area": floor["usable_area"] - floor["available_area"],
    "used_loading": floor["max_loading"] - floor["available_loading"]
}for floor in all_floors]
assignment_summary, unassigned_blocks



✅ All blocks assigned successfully.

✅ All blocks assigned successfully.


([{'floor': 'L001Ground Floor',
   'max_loading': 160,
   'usable_area': 1595,
   'used_area': 0,
   'used_loading': 0},
  {'floor': 'L002Floor 01',
   'max_loading': 120,
   'usable_area': 1181,
   'used_area': 0,
   'used_loading': 0},
  {'floor': 'L003Floor 02',
   'max_loading': 225,
   'usable_area': 2363,
   'used_area': 0,
   'used_loading': 0},
  {'floor': 'L004Floor 03',
   'max_loading': 225,
   'usable_area': 2363,
   'used_area': 0,
   'used_loading': 0},
  {'floor': 'L005Floor 04',
   'max_loading': 225,
   'usable_area': 2363,
   'used_area': 0,
   'used_loading': 0},
  {'floor': 'L006Floor 05',
   'max_loading': 225,
   'usable_area': 2363,
   'used_area': 0,
   'used_loading': 0},
  {'floor': 'L007Floor 06',
   'max_loading': 225,
   'usable_area': 2363,
   'used_area': 0,
   'used_loading': 0},
  {'floor': 'L008Floor 07',
   'max_loading': 225,
   'usable_area': 2327,
   'used_area': 0,
   'used_loading': 0},
  {'floor': 'L009Floor 08',
   'max_loading': 210,
   'usabl

# Typical Final

In [ ]:
import random
from collections import Counter, defaultdict

typical_blocks = [
    block for block in all_block_data
    if str(block.get("Typical/Destination", "")).strip().lower() in ["typical"]
]


# Calculate SpaceMix ratios
space_mix_values = [block.get("SpaceMix ( ME/ WE/ US/ Support/ Speciality)", "Unknown").strip() for block in typical_blocks]
space_mix_counts = Counter(space_mix_values)
total_typical = len(typical_blocks)
space_mix_ratios = {cat: count / total_typical for cat, count in space_mix_counts.items()}

# Group blocks by SpaceMix type
blocks_by_spacemix = defaultdict(list)
for block in typical_blocks:
    key = block.get("SpaceMix ( ME/ WE/ US/ Support/ Speciality)", "Unknown").strip()
    blocks_by_spacemix[key].append(block)

# Shuffle floors for random distribution


# Config
batch_size = 10


# Form and assign full batches using ratio
def get_full_batch(ratios, grouped_blocks, batch_size):
    batch = []
    selected_counts = {}

    for space_type, ratio in ratios.items():
        needed = int(round(ratio * batch_size))
        available = grouped_blocks[space_type]

        if len(available) < needed:
            return None  # Cannot form a valid batch

        selected = random.sample(available, needed)  # <--- Random selection here
        batch.extend(selected)
        selected_counts[space_type] = selected

    # Remove selected blocks from their categories
    for space_type, selected in selected_counts.items():
        for sel_block in selected:
            grouped_blocks[space_type].remove(sel_block)

    random.shuffle(batch)
    return batch

import random

def assign_batch_to_floors(batch, floors):
    # Shuffle floors to randomize distribution
    random.shuffle(floors)

    area_needed = sum(block.get("Cumulative Block + Circulation Area (SQM)", 0) for block in batch)
    loading_needed = sum(block.get("Max Occupancy by Block", 0) * block.get("Block Capacity", 0) for block in batch)

    for floor in floors:
        if area_needed <= floor["available_area"] and loading_needed <= floor["available_loading"]:
            floor["assigned_blocks"].extend(batch)
            floor["available_area"] -= area_needed
            floor["available_loading"] -= loading_needed
            return True
    return False


# Assign full batches
while True:
    batch = get_full_batch(space_mix_ratios, blocks_by_spacemix, batch_size)
    if not batch:
        break
    if not assign_batch_to_floors(batch, all_floors):
        unassigned_blocks.extend(batch)


# Gather all leftover blocks into one list
for blocks in blocks_by_spacemix.values():
    unassigned_blocks.extend(blocks)

# Assign remaining blocks one by one
random.shuffle(unassigned_blocks)
final_unassigned = []

for block in unassigned_blocks:
    assigned = False
    area_needed = block.get("Cumulative Block + Circulation Area (SQM)", 0)
    loading_needed = block.get("Max Occupancy by Block", 0) * block.get("Block Capacity", 0)

    for floor in all_floors:
        if area_needed <= floor["available_area"] and loading_needed <= floor["available_loading"]:
            floor["assigned_blocks"].append(block)
            floor["available_area"] -= area_needed
            floor["available_loading"] -= loading_needed
            assigned = True
            break

    if not assigned:
        final_unassigned.append(block)

# Replace old list with only those still unassigned
unassigned_blocks = final_unassigned

# Prepare output
assignment_summary = [{
    "floor": floor["name"],
   # "assigned_blocks": [block["Block Name"] for block in floor["assigned_blocks"]],
    "max_loading": floor["max_loading"],
    "usable_area": floor["usable_area"],
    "used_area": floor["usable_area"] - floor["available_area"],
    "used_loading": floor["max_loading"] - floor["available_loading"]
}for floor in all_floors]

assignment_summary




[{'floor': 'L001Ground Floor',
  'max_loading': 160,
  'usable_area': 1595,
  'used_area': 0,
  'used_loading': 0},
 {'floor': 'L002Floor 01',
  'max_loading': 120,
  'usable_area': 1181,
  'used_area': 0,
  'used_loading': 0},
 {'floor': 'L003Floor 02',
  'max_loading': 225,
  'usable_area': 2363,
  'used_area': 0,
  'used_loading': 0},
 {'floor': 'L004Floor 03',
  'max_loading': 225,
  'usable_area': 2363,
  'used_area': 0,
  'used_loading': 0},
 {'floor': 'L005Floor 04',
  'max_loading': 225,
  'usable_area': 2363,
  'used_area': 0,
  'used_loading': 0},
 {'floor': 'L006Floor 05',
  'max_loading': 225,
  'usable_area': 2363,
  'used_area': 0,
  'used_loading': 0},
 {'floor': 'L007Floor 06',
  'max_loading': 225,
  'usable_area': 2363,
  'used_area': 0,
  'used_loading': 0},
 {'floor': 'L008Floor 07',
  'max_loading': 225,
  'usable_area': 2327,
  'used_area': 0,
  'used_loading': 0},
 {'floor': 'L009Floor 08',
  'max_loading': 210,
  'usable_area': 2281,
  'used_area': 0,
  'used_lo

# Unassigned Blocks

# Excel File Creation (Assigned)


In [ ]:
import pandas as pd

# Prepare rows for detailed assignment
rows = []

# Prepare dictionary for floor summary
floor_summary = {}

# Sort floors by 'name' in ascending order before processing
for floor in sorted(all_floors, key=lambda f: f["name"]):
    floor_name = floor["name"]
    total_area = 0
    total_max_occ = 0
    total_occ_used = 0

    for block in floor["assigned_blocks"]:
        block_area = block.get("Cumulative Block + Circulation Area (SQM)", 0)
        block_capacity = block.get("Block Capacity", 0)
        block_max_occ = block.get("Max Occupancy by Block", 0) * block_capacity

        total_area += block_area
        total_max_occ += block_capacity  # Assuming capacity is the max occupancy possible
        total_occ_used += block.get("Max Occupancy by Block", 0)

        rows.append({
            "Level": floor_name,
            "Block": block.get("Block Name", "Unnamed Block"),
            "Department_Sub-Department": block.get("Department_Sub-Department", ""),
            "Area (including circulation)": block_area,
            "Max Occupancy by Block": block_max_occ,
            "Spacemix": block.get("SpaceMix ( ME/ WE/ US/ Support/ Speciality)", ""),
            "Typical/Destination": block.get("Typical/Destination", "")
        })

    # Store floor summary
    floor_summary[floor_name] = {
        "Floor": floor_name,
        "Total Area Used (SQM)": total_area,
        "Total Max Occupancy": total_max_occ,
        "Total Occupancy Used": total_occ_used
    }

# Create DataFrames
df_details = pd.DataFrame(rows)
df_summary = pd.DataFrame(floor_summary.values())

# Save both sheets to Excel
output_file = "assigned_blocks_by_floor.xlsx"
with pd.ExcelWriter(output_file) as writer:
    df_details.to_excel(writer, sheet_name="Detailed Assignments", index=False)
    df_summary.to_excel(writer, sheet_name="Floor Summary", index=False)

# Download in Google Colab
from google.colab import files
files.download(output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Excel Unassigned

In [ ]:
import pandas as pd

# Prepare rows for unassigned blocks
unassigned_rows = []

for block in final_unassigned:
    unassigned_rows.append({
        "Block": block.get("Block Name", "Unnamed Block"),
        "Department_Sub-Department": block.get("Department_sub_department", ""),
        "Area (including circulation)": block.get("Cumulative Block + Circulation Area (SQM)", 0),
        "Max Occupancy by Block": block.get("Max Occupancy by Block", 0) * block.get("Block Capacity", 0),
        "Spacemix": block.get("SpaceMix ( ME/ WE/ US/ Support/ Speciality)", ""),
        "Typical/Destination": block.get("Typical/Destination", "")
    })

# Create DataFrame
df_unassigned = pd.DataFrame(unassigned_rows)

# Save to Excel
unassigned_file = "unassigned_blocks.xlsx"
df_unassigned.to_excel(unassigned_file, index=False)

# Download in Colab
from google.colab import files
files.download(unassigned_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>